# Week 5 Homework

***Due (pushed to your GitHub branch) on 10/18 by 11:59 pm***

## Snakemake Pipeline

In contrast to last week (FASTQ QC), there aren't as many decisions that need to be made during alignment, so using a notebook to control and report code isn't as necessary. As you saw in in [5_alignment](5_alignment.ipynb), file and directory management can get complicated during alignment. This is a case where Snakemake scripting shines.

1. Copy your `Snakefile` and `config.yaml` from `4_fastq` to `5_alignment`. 
2. In the `Snakefile`, produce the following rules:  
    a. `fetch_genome` - using a variation from the code from `2_genome_exploration`, download your reference genome and annotations.
    b. `star_sa` - use `STAR` to produce the suffix array of the reference genome.  
    c. `star_align_first` - use `STAR` to perform first-pass alignment with all FASTQ files.
    d. `star_align_second` - use `STAR` to perform second-pass alignment with all FASTQ files.
    e. `hisat_algin` - use `HISAT` to align all FASTQ files.
    e. `report` - use `multiqc` to aggregate all QC data into a single report.

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`.